In [68]:
import numpy as np
import random
import pandas as pd

In [69]:
data = pd.read_csv("datasets/NationalNames.csv")
data.drop(['Id','Year','Count'], axis = 1)
data.head()

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746


In [70]:
male = data.loc[data['Gender'] == 'M']['Name']
female = data.loc[data['Gender'] == 'F']['Name']
male.head()

942       John
943    William
944      James
945    Charles
946     George
Name: Name, dtype: object

In [71]:
female = list(female)
np.random.shuffle(female)
for i in range(len(female)):
    female[i] = female[i] + '\n'
female = [m.lower() for m in female]
female = female[:200]
data_size = len(female)
chars = set(''.join(female))
char_size = len(chars)
print("Data size {}\nUnique characters: {}".format(data_size, char_size))
m = len(female)
max_len = 30

Data size 200
Unique characters: 27


In [72]:
char_to_ix = {ch:i for i,ch in enumerate(sorted(chars))}
ix_to_char = {i:ch for i,ch in enumerate(sorted(chars))}

In [73]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Lambda
from keras.layers import LSTM
from numpy import array
from keras.optimizers import Adam

In [74]:
X = []
Y = []
for i in range(m):
    x = [char_to_ix[ch] for ch in female[i]]
    x.extend([0] * (max_len - len(x)))
    
    y = x[1:] + [char_to_ix["\n"]]
    X.append(x)
    Y.append(y)

X = keras.utils.to_categorical(X, num_classes = char_size, dtype = 'float32')
Y = keras.utils.to_categorical(Y, num_classes = char_size, dtype = 'float32')
Y = np.swapaxes(Y,0,1)

print(X.shape)
print(Y.shape)

(200, 30, 27)
(30, 200, 27)


In [75]:
n_a = 128
Tx = 30
reshapor = Reshape((1, char_size))
LSTM_cell = LSTM(n_a, return_state = True)
densor = Dense(char_size, activation = 'softmax')

In [76]:
def train_model(Tx, n_a, char_size):
    X = Input(shape = (Tx, char_size))
    a0 = Input(shape = (n_a,))
    c0 = Input(shape = (n_a,))
    
    a = a0
    c = c0
    
    outputs = []
    for t in range(Tx):
        x = Lambda(lambda x: X[:,t,:])(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(x, initial_state = [a,c])
        out = densor(a)
        outputs.append(out)
    model = Model([X, a0, c0], outputs)
    return model

In [77]:
model = train_model(Tx, n_a, char_size)

In [78]:
a0 = np.zeros((X.shape[0], n_a))
c0 = np.zeros((X.shape[0], n_a))
output = model.predict([X, a0, c0])

In [79]:
print(np.array(output).shape)

(30, 200, 27)


In [80]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [81]:
a0 = np.zeros((X.shape[0], n_a))
c0 = np.zeros((X.shape[0], n_a))
model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
200/200 [==============================] - 32s 159ms/step - loss: 44.9082 - dense_2_loss: 0.8761 - dense_2_acc: 0.0050 - dense_2_acc_1: 0.0100 - dense_2_acc_2: 0.0200 - dense_2_acc_3: 0.0950 - dense_2_acc_4: 0.2700 - dense_2_acc_5: 0.5050 - dense_2_acc_6: 0.6850 - dense_2_acc_7: 0.7900 - dense_2_acc_8: 0.8150 - dense_2_acc_9: 0.8250 - dense_2_acc_10: 0.8350 - dense_2_acc_11: 0.8350 - dense_2_acc_12: 0.8350 - dense_2_acc_13: 0.8350 - dense_2_acc_14: 0.8400 - dense_2_acc_15: 0.8400 - dense_2_acc_16: 0.8400 - dense_2_acc_17: 0.8400 - dense_2_acc_18: 0.8400 - dense_2_acc_19: 0.8400 - dense_2_acc_20: 0.8400 - dense_2_acc_21: 0.8400 - dense_2_acc_22: 0.8400 - dense_2_acc_23: 0.8400 - dense_2_acc_24: 0.8400 - dense_2_acc_25: 0.8400 - dense_2_acc_26: 0.8400 - dense_2_acc_27: 0.8400 - dense_2_acc_28: 0.8400 - dense_2_acc_29: 0.8400                                                                                                        
Epoch 2/100
200/200 [============================

200/200 [==============================] - 0s 2ms/step - loss: 16.0057 - dense_2_loss: 7.2066e-04 - dense_2_acc: 0.2650 - dense_2_acc_1: 0.1200 - dense_2_acc_2: 0.1700 - dense_2_acc_3: 0.2350 - dense_2_acc_4: 0.3450 - dense_2_acc_5: 0.6000 - dense_2_acc_6: 0.8000 - dense_2_acc_7: 0.9450 - dense_2_acc_8: 0.9750 - dense_2_acc_9: 0.9850 - dense_2_acc_10: 0.9950 - dense_2_acc_11: 0.9950 - dense_2_acc_12: 0.9950 - dense_2_acc_13: 0.9950 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 1.0000
Epoch 11/100
200/200 [==============================] - 0s 2ms/step - loss: 15.9508 - dense_2_loss: 5.0554e-04 - dense_2_acc: 0.2650 - dense_2_acc_1: 0.1250 - dense

200/200 [==============================] - 0s 2ms/step - loss: 14.0476 - dense_2_loss: 1.0354e-04 - dense_2_acc: 0.3450 - dense_2_acc_1: 0.2350 - dense_2_acc_2: 0.2600 - dense_2_acc_3: 0.2950 - dense_2_acc_4: 0.4350 - dense_2_acc_5: 0.6400 - dense_2_acc_6: 0.8050 - dense_2_acc_7: 0.9500 - dense_2_acc_8: 0.9750 - dense_2_acc_9: 0.9850 - dense_2_acc_10: 0.9950 - dense_2_acc_11: 0.9950 - dense_2_acc_12: 0.9950 - dense_2_acc_13: 0.9950 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 1.0000
Epoch 20/100
200/200 [==============================] - 0s 2ms/step - loss: 13.8648 - dense_2_loss: 7.8036e-05 - dense_2_acc: 0.3450 - dense_2_acc_1: 0.2400 - dense

200/200 [==============================] - 0s 2ms/step - loss: 12.6036 - dense_2_loss: 2.7686e-05 - dense_2_acc: 0.3450 - dense_2_acc_1: 0.2700 - dense_2_acc_2: 0.3350 - dense_2_acc_3: 0.3650 - dense_2_acc_4: 0.5050 - dense_2_acc_5: 0.6550 - dense_2_acc_6: 0.8200 - dense_2_acc_7: 0.9500 - dense_2_acc_8: 0.9750 - dense_2_acc_9: 0.9850 - dense_2_acc_10: 0.9950 - dense_2_acc_11: 0.9950 - dense_2_acc_12: 0.9950 - dense_2_acc_13: 0.9950 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 1.0000
Epoch 29/100
200/200 [==============================] - 0s 2ms/step - loss: 12.4356 - dense_2_loss: 2.7470e-05 - dense_2_acc: 0.3450 - dense_2_acc_1: 0.2750 - dense

200/200 [==============================] - 0s 2ms/step - loss: 11.4774 - dense_2_loss: 1.4611e-05 - dense_2_acc: 0.3600 - dense_2_acc_1: 0.2950 - dense_2_acc_2: 0.3650 - dense_2_acc_3: 0.4150 - dense_2_acc_4: 0.5400 - dense_2_acc_5: 0.6900 - dense_2_acc_6: 0.8400 - dense_2_acc_7: 0.9700 - dense_2_acc_8: 0.9850 - dense_2_acc_9: 0.9900 - dense_2_acc_10: 0.9950 - dense_2_acc_11: 0.9950 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 0.9950 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 1.0000
Epoch 38/100
200/200 [==============================] - 0s 2ms/step - loss: 11.3322 - dense_2_loss: 1.4228e-05 - dense_2_acc: 0.3600 - dense_2_acc_1: 0.3100 - dense

...

200/200 [==============================] - 0s 2ms/step - loss: 8.5707 - dense_2_loss: 8.2866e-06 - dense_2_acc: 0.3650 - dense_2_acc_1: 0.3400 - dense_2_acc_2: 0.4750 - dense_2_acc_3: 0.6000 - dense_2_acc_4: 0.6650 - dense_2_acc_5: 0.8700 - dense_2_acc_6: 0.9450 - dense_2_acc_7: 0.9850 - dense_2_acc_8: 0.9950 - dense_2_acc_9: 0.9950 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 1.0000
Epoch 65/100
200/200 [==============================] - 0s 2ms/step - loss: 8.4881 - dense_2_loss: 7.5651e-06 - dense_2_acc: 0.3650 - dense_2_acc_1: 0.3550 - dense_2

200/200 [==============================] - 0s 2ms/step - loss: 7.7359 - dense_2_loss: 7.2098e-06 - dense_2_acc: 0.3650 - dense_2_acc_1: 0.3450 - dense_2_acc_2: 0.5050 - dense_2_acc_3: 0.6750 - dense_2_acc_4: 0.7750 - dense_2_acc_5: 0.9400 - dense_2_acc_6: 0.9550 - dense_2_acc_7: 0.9850 - dense_2_acc_8: 1.0000 - dense_2_acc_9: 1.0000 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 1.0000
Epoch 74/100
200/200 [==============================] - 0s 2ms/step - loss: 7.6892 - dense_2_loss: 6.3783e-06 - dense_2_acc: 0.3700 - dense_2_acc_1: 0.3400 - dense_2

200/200 [==============================] - 0s 2ms/step - loss: 7.1300 - dense_2_loss: 6.7693e-06 - dense_2_acc: 0.3700 - dense_2_acc_1: 0.3600 - dense_2_acc_2: 0.5350 - dense_2_acc_3: 0.7050 - dense_2_acc_4: 0.8600 - dense_2_acc_5: 0.9400 - dense_2_acc_6: 0.9700 - dense_2_acc_7: 0.9950 - dense_2_acc_8: 1.0000 - dense_2_acc_9: 1.0000 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 1.0000
Epoch 83/100
200/200 [==============================] - 0s 2ms/step - loss: 7.0761 - dense_2_loss: 7.0897e-06 - dense_2_acc: 0.3700 - dense_2_acc_1: 0.3700 - dense_2

200/200 [==============================] - 0s 2ms/step - loss: 6.5937 - dense_2_loss: 6.5324e-06 - dense_2_acc: 0.3700 - dense_2_acc_1: 0.3700 - dense_2_acc_2: 0.5750 - dense_2_acc_3: 0.7750 - dense_2_acc_4: 0.9100 - dense_2_acc_5: 0.9750 - dense_2_acc_6: 0.9950 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - dense_2_acc_9: 1.0000 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 1.0000
Epoch 92/100
200/200 [==============================] - 0s 2ms/step - loss: 6.5364 - dense_2_loss: 6.3527e-06 - dense_2_acc: 0.3700 - dense_2_acc_1: 0.3750 - dense_2

200/200 [==============================] - 0s 2ms/step - loss: 6.1601 - dense_2_loss: 5.7784e-06 - dense_2_acc: 0.3800 - dense_2_acc_1: 0.3850 - dense_2_acc_2: 0.6100 - dense_2_acc_3: 0.8150 - dense_2_acc_4: 0.9400 - dense_2_acc_5: 0.9900 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 1.0000 - dense_2_acc_8: 1.0000 - dense_2_acc_9: 1.0000 - dense_2_acc_10: 1.0000 - dense_2_acc_11: 1.0000 - dense_2_acc_12: 1.0000 - dense_2_acc_13: 1.0000 - dense_2_acc_14: 1.0000 - dense_2_acc_15: 1.0000 - dense_2_acc_16: 1.0000 - dense_2_acc_17: 1.0000 - dense_2_acc_18: 1.0000 - dense_2_acc_19: 1.0000 - dense_2_acc_20: 1.0000 - dense_2_acc_21: 1.0000 - dense_2_acc_22: 1.0000 - dense_2_acc_23: 1.0000 - dense_2_acc_24: 1.0000 - dense_2_acc_25: 1.0000 - dense_2_acc_26: 1.0000 - dense_2_acc_27: 1.0000 - dense_2_acc_28: 1.0000 - dense_2_acc_29: 1.0000


In [83]:
x_initializer = np.zeros((1, 1, char_size))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

In [84]:
def generate_model(char_size, n_a, Ty):
    x0 = Input(shape=(1, char_size))
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0
    outputs = []
    for t in range(Ty):
        a, _, c = LSTM_cell(x, initial_state = [a,c])
        out = densor(a)
        outputs.append(out)
    inference_model = Model([x0, a0, c0], outputs)
    return inference_model


In [95]:
inference_model = generate_model(char_size, n_a = n_a, Ty = 30)

In [98]:
x_initializer = np.random.randn(1, 1, char_size)
a_initializer = np.random.randn(1, n_a)
c_initializer = np.random.randn(1, n_a)
pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
for item in pred:
    onehot = item[0]
    id = np.argmax(onehot)
    print(ix_to_char[id], end="")

ii



























